<a href="https://colab.research.google.com/github/Aya-Jafar/Traffic-jam-prediction/blob/main/Climate_capstone_project_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn 
from sklearn import  model_selection, svm, metrics, pipeline, feature_selection, preprocessing
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
import tensorflow as tf
from tqdm import tqdm
from sklearn.metrics import mean_squared_error
from sklearn import feature_selection
import requests

In [ ]:
clean_df = pd.read_csv('/content/drive/MyDrive/last-but-not-least.csv')
clean_df.drop(columns=['Unnamed: 0'],inplace=True)
clean_df

,SPEED,STREET,DIRECTION,FROM_STREET,TO_STREET,LENGTH,MINUTE,BUS_COUNT,HOUR,DAY_OF_WEEK,...,END_FUEL_STATION_COUNT,END_PARKS_COUNT,END_SCHOOLS_COUNT,START_RESTURANTS_CAFE'S_COUNT,START_COLLEGE_COUNT,START_HOSPITALS_COUNT,START_FUEL_STATION_COUNT,START_PARKS_COUNT,START_SCHOOLS_COUNT,TIME_TAKEN
0,15.0,1,0,1,1,0.51,31,1,8,3,...,9,0,4,1,4,5,10,0,6,2.040000
1,27.0,3,0,4,4,0.50,20,1,12,3,...,11,1,3,3,2,1,12,1,3,1.111111
2,25.0,4,2,5,5,0.31,10,2,19,3,...,0,3,0,8,7,7,0,3,0,0.744000
3,27.0,5,2,8,8,0.72,30,1,5,2,...,5,3,2,0,2,1,9,2,0,1.600000
4,35.0,9,2,11,12,0.50,50,1,1,4,...,9,1,3,6,0,0,7,1,2,0.857143
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,23.0,43,2,114,51,0.41,1,0,3,2,...,8,1,4,3,1,2,10,1,4,1.069565
9996,32.0,3,0,71,70,0.50,50,0,1,3,...,8,2,8,1,0,3,8,1,9,0.937500
9997,25.0,20,2,52,51,0.50,30,0,1,4,...,1,3,0,3,9,9,5,4,0,1.200000
9998,18.0,0,0,60,59,0.50,1,0,6,6,...,4,2,2,4,3,0,4,3,2,1.666667


In [ ]:
clean_df.drop(columns=['TIME_TAKEN'],inplace=True)

In [ ]:
clean_df.drop(columns=['DIRECTION'],inplace=True)

In [ ]:
TIME_TAKEN = []
for i in clean_df[['LENGTH','SPEED']].to_numpy():
    if i[1] == 0:
      TIME_TAKEN.append(np.nan)
    else:
      TIME_TAKEN.append( (i[0] / i[1]) *60 )

clean_df['TIME_TAKEN'] = TIME_TAKEN
clean_df['TIME_TAKEN'].fillna(np.mean(clean_df['TIME_TAKEN']) ,inplace=True)

In [ ]:
clean_df['TIME_TAKEN'] = clean_df['TIME_TAKEN'].round(2) 

In [ ]:
x = clean_df.drop(columns=['SPEED'])
y = clean_df.SPEED

# Normalization 

In [ ]:
scaler = preprocessing.StandardScaler()
# Use the fit method to get mean and std from the dataset 
scaler.fit(x)
# Apply the stander scaler transformation on the dataset 
x_scaler = scaler.transform(x)

In [ ]:
import joblib
joblib.dump(scaler, 'scaler.joblib')

['scaler.joblib']

In [ ]:
y_normalize=preprocessing.MinMaxScaler()  
y_norm=y_normalize.fit_transform(y.to_numpy().reshape(-1,1))#normlize the y 
y_norm = y_norm.reshape(-1)

In [ ]:
joblib.dump(y_normalize, 'y_normalize.joblib')

['y_normalize.joblib']

# Feature selection

In [ ]:
clean_df.columns

Index(['SPEED', 'STREET', 'FROM_STREET', 'TO_STREET', 'LENGTH', 'MINUTE',
       'BUS_COUNT', 'HOUR', 'DAY_OF_WEEK', 'MONTH', 'DAY_OF_MONTH',
       'START_LATITUDE', 'START_LONGITUDE', 'END_LATITUDE', 'END_LONGITUDE',
       'END_RESTURANTS_CAFE'S_COUNT', 'END_COLLEGE_COUNT',
       'END_HOSPITALS_COUNT', 'END_FUEL_STATION_COUNT', 'END_PARKS_COUNT',
       'END_SCHOOLS_COUNT', 'START_RESTURANTS_CAFE'S_COUNT',
       'START_COLLEGE_COUNT', 'START_HOSPITALS_COUNT',
       'START_FUEL_STATION_COUNT', 'START_PARKS_COUNT', 'START_SCHOOLS_COUNT',
       'TIME_TAKEN'],
      dtype='object')

# Splitting the dataset

In [ ]:
x_train,x_test,y_train,y_test = model_selection.train_test_split(x_scaler,y_norm,test_size=0.2,random_state=1)

# ML models

In [ ]:
models = [
    sklearn.linear_model.LinearRegression() , 
    svm.SVR(kernel='rbf'),
    RandomForestRegressor(random_state=1),
    GradientBoostingRegressor(random_state=1),
    XGBRegressor()
]

mertic = {
    'Model':[],
    'Train_MSE':[],
    'Test_MSE':[]
}

for model in models:
  model.fit(x_train, y_train)
  train_pred = model.predict(x_train)
  train_mse = mean_squared_error(y_train, train_pred)
  mertic['Train_MSE'].append(train_mse)

  pred = model.predict(x_test)
  mse = metrics.mean_squared_error(y_test, pred)
  mertic['Model'].append(str(model).split('(')[0])
  mertic['Test_MSE'].append(mse)


mertic = pd.DataFrame(mertic)
mertic

,Model,Train_MSE,Test_MSE
0,LinearRegression,0.006275,0.004797
1,SVR,0.002735,0.002887
2,RandomForestRegressor,0.000025,0.000059
3,GradientBoostingRegressor,0.000235,0.000181
4,XGBRegressor,0.000006,0.000085


# Training ANN

In [ ]:
input_shape = x.shape[1]

model = tf.keras.Sequential(
    [
        # hidden layers
        tf.keras.layers.Dense(256, input_shape=(input_shape,),activation="relu"),
        tf.keras.layers.Dense(128,activation="relu"),
        tf.keras.layers.Dense(64,activation="relu"),
        # output layer
        tf.keras.layers.Dense(1),
    ]
)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               7168      
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 dense_2 (Dense)             (None, 64)                8256      
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 48,385
Trainable params: 48,385
Non-trainable params: 0
_________________________________________________________________


In [ ]:
loss_fn = tf.losses.mean_squared_error
optimizer = tf.optimizers.Adam()
metrics = [tf.keras.metrics.MeanSquaredError()]

# Compile the model
model.compile(optimizer=optimizer, loss=loss_fn, metrics=metrics)
model.fit(x_train, y_train, batch_size=32, epochs=10, validation_data=(x_test, y_test))

Epoch 1/10
250/250 [==============================] - 2s 5ms/step - loss: 0.0085 - mean_squared_error: 0.0085 - val_loss: 0.0028 - val_mean_squared_error: 0.0028
Epoch 2/10
250/250 [==============================] - 1s 3ms/step - loss: 0.0019 - mean_squared_error: 0.0019 - val_loss: 0.0013 - val_mean_squared_error: 0.0013
Epoch 3/10
250/250 [==============================] - 1s 3ms/step - loss: 0.0011 - mean_squared_error: 0.0011 - val_loss: 8.6351e-04 - val_mean_squared_error: 8.6351e-04
Epoch 4/10
250/250 [==============================] - 1s 3ms/step - loss: 7.0350e-04 - mean_squared_error: 7.0350e-04 - val_loss: 8.6316e-04 - val_mean_squared_error: 8.6316e-04
Epoch 5/10
250/250 [==============================] - 1s 3ms/step - loss: 6.4040e-04 - mean_squared_error: 6.4040e-04 - val_loss: 5.3378e-04 - val_mean_squared_error: 5.3378e-04
Epoch 6/10
250/250 [==============================] - 1s 3ms/step - loss: 4.4938e-04 - mean_squared_error: 4.4938e-04 - val_loss: 5.3462e-04 - val_mea

# Choosing the best K and Cross validation

In [ ]:
def train_test_split(x, y, train_indices, test_indices):
    return x[train_indices], x[test_indices], y[train_indices], y[test_indices]

In [ ]:
def cross_validation(model):

    result = {
        'MSE':[],
    }

    k_fold = model_selection.KFold(n_splits=10, shuffle=True, random_state=42)

    for train_indexes,test_indexes in k_fold.split(x_scaler,y_norm):

            x_train,x_test,y_train,y_test = train_test_split(x_scaler,y_norm,train_indexes,test_indexes)

            model.fit(x_train, y_train)

            # Predict using test set
            pred = model.predict(x_test)
            e = mean_squared_error(y_test,pred) 
            result['MSE'].append(e)
            
    return pd.DataFrame(result)
        
result = cross_validation( RandomForestRegressor(random_state=1) ) 
result

,MSE
0,0.000486
1,0.000183
2,0.000108
3,0.000068
4,0.000174
5,0.000099
6,0.000078
7,0.000069
8,0.000064
9,0.000063


In [ ]:
result.mean()

MSE    0.000139
dtype: float64

In [ ]:
result2 = cross_validation(
     XGBRegressor()
)

In [ ]:
result2.mean()

MSE    0.000128
dtype: float64

In [ ]:
joblib.dump(result2.mean()[0], 'final-result.joblib')

['final-result.joblib']

In [ ]:
!pip install gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.9/19.9 MB 46.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 295.8 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.1/288.1 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.6/136.6 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 

In [ ]:
import gradio as gr
import datetime

In [ ]:
final_chosen_model =  XGBRegressor()
final_chosen_model.fit(x_train,y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [ ]:
def search_places(ll , length):
  
    RADUIS = int(( length * 1600 ) // 2)

    CLIENT_ID = "S4FJMNBMU2ZEORB2EWZKAAU0UOPQX5LHR5SFXWQFDUXXWBYH"
    CLIENT_SECRET = "3L3EHFPASJQXDTVERNF44EBY1JBWNQOKHRB1FZQAQX2HXZG5"
    API_KEY = 'fsq399RuwPzWjtV+l+EsS5A3XxMqNPQHZFfeqmt4x+JYaSk='

    headers = {
          'Accept': 'application/json',
          'Authorization': API_KEY
    }

    VERSION = 20230316

    url =f'https://api.foursquare.com/v3/places/search?client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&v={VERSION}&ll={ll[0]},{ll[1]}&raduis={RADUIS}&query=restaurant,gas%20station,hospital,park,university,school&limit=50'


    response = requests.get(url,headers=headers)
    # print(response)

    places_counts = {
        'Resturants and Cafe\'s': 0,
        'College and University':0,
        'Hospitals':0,
        'Fuel station':0,
        'Parks':0,
        'Schools':0,
    }
    data = response.json()
    # print(data)
    # print(data['results'][0]['categories'])
   
    resturants_ids = [13000 , 13032 , 13037, 13065, 10020 , 10018 ,13034, 13035,13067, 13068, 13073, 13072]
    college_and_unis_ids = [12009 , 12013 , 12047 ]
    hospitals_ids = [15014 , 15000,15016 , 15033 , 15032 ,15045, 17035 ]
    fuel_stations_ids = [ 19007 ]
    parks_ids = [12000 , 19020 , 16032 ]
    schools_ids = [12009 , 12058 , 12057 , 12059 , 12060]


    try:
      if len(data['results']) :
            for i in data['results']:
              # print(i)
              if (len(i['categories'])):
                  if i['categories'][0]['id'] in resturants_ids:
                      places_counts['Resturants and Cafe\'s'] += 1

                  elif i['categories'][0]['id'] in college_and_unis_ids:
                    places_counts['College and University'] += 1
                  
                  elif i['categories'][0]['id'] in hospitals_ids:
                    places_counts['Hospitals'] += 1
                  
                  elif i['categories'][0]['id'] in fuel_stations_ids:
                    places_counts['Fuel station'] += 1
                  
                  elif i['categories'][0]['id'] in  parks_ids:
                    places_counts['Parks'] += 1

                  elif i['categories'][0]['id'] in schools_ids:
                    places_counts['Schools'] += 1
      else:
        return places_counts
    except:
        return np.nan
      
    return places_counts

In [ ]:
# search_places([41.801585,	-87.645536		] , 0.50)

In [ ]:

encoded_street ={'Halsted': 0,
 'Cicero': 1,
 'Irving Park': 2,
 'Western': 3,
 'Cermak': 4,
 'Garfield': 5,
 'Pulaski': 6,
 'Dr Martin L King Jr': 7,
 'Columbus': 8,
 'Archer': 9,
 '127th': 10,
 'Lake Shore Dr': 11,
 'Michigan': 12,
 'Washington': 13,
 'Stony Island': 14,
 'Peterson': 15,
 '87th': 16,
 '55th': 17,
 '79th': 18,
 'Chicago': 19,
 'Roosevelt': 20,
 '95th': 21,
 'LaSalle': 22,
 'Ashland': 23,
 'Wacker': 24,
 'Randolph': 25,
 'Cumberland': 26,
 'Clark': 27,
 'Touhy': 28,
 'Illinois': 29,
 'State': 30,
 'Pershing': 31,
 'North': 32,
 'Harlem': 33,
 'Caldwell': 34,
 'Grand': 35,
 'Jackson': 36,
 'Harrison': 37,
 'Milwaukee': 38,
 'Ontario': 39,
 '130th': 40,
 'Madison': 41,
 'Ohio': 42,
 '111th': 43,
 'Torrence': 44,
 'Ewing': 45,
 'Desplaines': 46,
 '57th': 47,
 'Fairbanks': 48,
 'Higgins': 49,
 'Broadway': 50,
 'Ridge': 51,
 'Congress': 52,
 'Jefferson': 53,
 'Elston': 54,
 'Monterey': 55,
 'Ramp To Cermak Rd': 56,
 'Sheridan': 57,
 'Warren': 58,
 'Indiana': 59,
 'Balbo': 60,
 'Hollywood': 61}

bystreet_encoded = {'51st': 0,
 'Kinzie': 1,
 '107th': 2,
 'Kostner': 3,
 '63rd': 4,
 'Michigan': 5,
 'City Boundary': 6,
 'Stewart': 7,
 'Halsted': 8,
 '111th': 9,
 'Randolph': 10,
 'Harlem': 11,
 '43rd': 12,
 'Ohio': 13,
 'Racine': 14,
 '75th': 15,
 'Dr Martin L King Jr': 16,
 'California': 17,
 '91st': 18,
 'Central Park': 19,
 'LaSalle': 20,
 'Bryn Mawr': 21,
 'Cicero': 22,
 '55th': 23,
 'Foster': 24,
 'Roosevelt': 25,
 'Grand': 26,
 'Harrison': 27,
 'Central': 28,
 'Pershing': 29,
 'Lake Shore Dr': 30,
 '76th': 31,
 'Pulaski': 32,
 'Irving Park': 33,
 'Damen': 34,
 '59th': 35,
 'Kennedy Expy': 36,
 'Washington': 37,
 'Ridge': 38,
 'Oriole': 39,
 '47th': 40,
 'Stony Island': 41,
 'Wacker': 42,
 'Caldwell': 43,
 'Cermak': 44,
 '35th': 45,
 'Woodlawn': 46,
 'Kedzie': 47,
 'North Ave': 48,
 'Touhy': 49,
 '71st': 50,
 '67th': 51,
 'Ashland': 52,
 'Wentworth': 53,
 'Clark': 54,
 '79th': 55,
 'Laramie': 56,
 'Fullerton': 57,
 'Devon': 58,
 'Division': 59,
 'Addison': 60,
 'Columbus': 61,
 'Chicago': 62,
 '99th': 63,
 'Canal': 64,
 'Elston': 65,
 'Armitage': 66,
 'Montrose': 67,
 'Lake': 68,
 'Diversey': 69,
 'Western': 70,
 '87th': 71,
 'I-94 Expy': 72,
 'Archer': 73,
 'Sunnyside': 74,
 'Peterson': 75,
 'Oak St': 76,
 '18th': 77,
 'Belmont': 78,
 '31st': 79,
 '103rd': 80,
 'State': 81,
 '96th': 82,
 '83rd': 83,
 'Lawrence': 84,
 'Cottage Grove': 85,
 'Torrence': 86,
 'Congress': 87,
 'I-55 Expy': 88,
 'Columbus Ave': 89,
 'Indianapolis': 90,
 'Franklin': 91,
 '95th': 92,
 'Dan Ryan Expy': 93,
 'Crawford': 94,
 'Oak Park': 95,
 'Ridgeland': 96,
 '26th': 97,
 'Hollywood': 98,
 'Indiana': 99,
 'Orleans': 100,
 '57th': 101,
 'Madison': 102,
 'Narragansett': 103,
 'Austin': 104,
 'Clarendon': 105,
 'Edens Expy': 106,
 'Pratt': 107,
 'Van Buren': 108,
 'Cumberland': 109,
 'Monroe': 110,
 'Sheridan': 111,
 'Milwaukee': 112,
 '16th': 113,
 'Vincennes': 114,
 'Bishop Ford Expy': 115,
 'Illinois': 116,
 'Monterey': 117,
 'Balbo': 118,
 '112th': 119,
 'Forest Preserve Ave': 120,
 'Fairbanks': 121,
 'Grand Ave': 122,
 'Lake Shore Dr Michigan Exit Ramp': 123,
 'Jefferson': 124,
 'Garfield': 125,
 'Gunnison': 126,
 '18TH': 127,
 '24th': 128,
 'Trumball': 129,
 '1550 West': 130,
 'Oakwood': 131,
 '127th': 132,
 'Stockton': 133,
 '72nd': 134,
 '130th': 135,
 'Nagle': 136,
 'Jersey': 137,
 'Pacific': 138,
 'Jeffery': 139,
 'Ogden': 140,
 'Mendota': 141,
 'Chicago Skyway': 142,
 'LakeShore Dr': 143,
 'Jackson': 144}

end_bystreet_encoded = {'55th': 0,
 'Madison': 1,
 '103rd': 2,
 'Pulaski': 3,
 '67th': 4,
 'Dr Martin L King Jr': 5,
 'Cicero': 6,
 'Halsted': 7,
 'Dan Ryan Expy': 8,
 '107th': 9,
 '47th': 10,
 'Illinois': 11,
 'Oak Park': 12,
 'Indiana': 13,
 'Lake Shore Dr Michigan Entrance Ramp': 14,
 '71st': 15,
 'Western': 16,
 '87th': 17,
 'Kedzie': 18,
 'Peterson': 19,
 'Laramie': 20,
 'Ashland': 21,
 '51st': 22,
 'Lawrence': 23,
 'Harrison': 24,
 'Wacker': 25,
 'Washington': 26,
 'Austin': 27,
 '43rd': 28,
 'Michigan': 29,
 '79th': 30,
 'Columbus': 31,
 'Kostner': 32,
 'Montrose': 33,
 '18th': 34,
 'Roosevelt': 35,
 '57th': 36,
 'Van Buren': 37,
 'Harlem': 38,
 'Cottage Grove': 39,
 'Chicago': 40,
 'Lehigh': 41,
 '26th': 42,
 'Pershing': 43,
 'Foster': 44,
 '59th': 45,
 'Central Park': 46,
 'Archer': 47,
 'Armitage': 48,
 'Pratt': 49,
 '63rd': 50,
 'Racine': 51,
 'Addison': 52,
 '72nd': 53,
 '16th': 54,
 'Sunnyside': 55,
 'Diversey': 56,
 'Mendota': 57,
 'Ridgeland': 58,
 'Belmont': 59,
 'Division': 60,
 'Oriole': 61,
 '95th': 62,
 'Michigain': 63,
 'State': 64,
 'Grand': 65,
 'Fullerton': 66,
 'Congress': 67,
 'Irving Park': 68,
 'Damen': 69,
 '91st': 70,
 '31st': 71,
 'Devon': 72,
 'Franklin': 73,
 'Garfield': 74,
 'Fairbanks': 75,
 '75th': 76,
 'Forest Preserve Ave': 77,
 'North Ave': 78,
 'Elston': 79,
 'Cermak': 80,
 'Milwaukee': 81,
 '83rd': 82,
 '35th': 83,
 'Columbus Ave': 84,
 'Chicago Skyway': 85,
 'Stevenson Expy': 86,
 'California': 87,
 'Clark': 88,
 'Monroe': 89,
 'Crawford': 90,
 'Indianapolis': 91,
 'Stewart': 92,
 'Lake Shore Dr': 93,
 'I-94 Expy': 94,
 'City Boundary': 95,
 'I-55 Expy': 96,
 'Randolph': 97,
 '96th': 98,
 'Jeffery': 99,
 '111th': 100,
 '1550 West': 101,
 'Lake': 102,
 'Narragansett': 103,
 'Hollywood': 104,
 'Ogden': 105,
 'Central': 106,
 'Vincennes': 107,
 'LaSalle': 108,
 'Gunnison': 109,
 'Stockton': 110,
 'Oakwood': 111,
 'Kennedy Expy': 112,
 'Oak St': 113,
 'Sheridan': 114,
 'Woodlawn': 115,
 '112th': 116,
 'Bryn Mawr': 117,
 'Touhy': 118,
 'Ridge': 119,
 'Pacific': 120,
 '99th': 121,
 'Balbo': 122,
 'Clarendon': 123,
 'Trumball': 124,
 '76th': 125,
 'Stony Island': 126,
 'Wentworth': 127,
 'Canal': 128,
 'Ohio': 129,
 'Kinzie': 130,
 'Nagle': 131,
 'Bishop Ford Expy': 132,
 'Jefferson': 133,
 '24th': 134,
 'Jersey': 135,
 '130th': 136,
 'Monterey': 137,
 'Jackson': 138,
 'Torrence': 139,
 '127th': 140,
 'Edens Expy': 141,
 'Mobile': 142,
 'Cumberland': 143}

days = {'Sunday': 1, 'Monday': 2, 'Tuesday': 3, 'Wednesday': 4, 'Thursday': 5, 'Friday': 6, 'Saturday': 7}

# Deployment using Gradio

In [ ]:
days_of_week = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']
choices = [(day) for day in days_of_week]

inputs = [
    gr.inputs.Textbox(label='Main street name'),
    # gr.inputs.Dropdown(label='Main street name', choices=street_choices),

    gr.inputs.Textbox(label='Start bystreet name'),
    gr.inputs.Textbox(label='End bystreet name'),
    gr.inputs.Dropdown(label="Day in week" ,choices=choices,  default='Sunday'),

    gr.inputs.Number(label="Length of the trip (in miles)", default=0 ),
    gr.inputs.Slider(minimum= 0 , maximum= 59,  label="Time taken for the trip (in mintues)",step=0.01 ,default=0 ),
    gr.inputs.Number(label="Bus count", default=0 ),

    gr.inputs.Slider(minimum= -90 , maximum= 90, label="Start Latidute", step=0.000001 ),
    gr.inputs.Slider(minimum= -180 , maximum= 180 , label="Start Longitude", step=0.000001),

    
    gr.inputs.Slider(minimum= -90 , maximum= 90, label="End Latidute", step=0.000001),
    gr.inputs.Slider(minimum= -180 , maximum= 180 , label="End Longitude", step=0.000001),

]

output = gr.outputs.Textbox()

/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:27: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:30: UserWarning: `optional` parameter is deprecated, and it has no effect
  super().__init__(
/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:30: UserWarning: `numeric` parameter is deprecated, and it has no effect
  super().__init__(
/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:219: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:222: UserWarning: `optional` parameter is deprecated, and it has no effect
  super().__init__(
/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:59: UserWarning: Usage of gradio.inputs is de

In [ ]:
def predict_congestion(main_street, start_street, end_street, day_in_week, length, time_taken, bus_count, start_lat, start_long, end_lat, end_long):

    # Calculate the current datetime
    min =  datetime.datetime.now().minute
    hour = datetime.datetime.now().hour
    month = datetime.datetime.now().month
    day_in_month = datetime.datetime.now().day
    
    # Calculate places count in the end coordination 
    e_resturants_count = search_places([ end_lat , end_long] , length)['Resturants and Cafe\'s']
    e_college_count = search_places([ end_lat , end_long] , length)['College and University']
    e_hospitals_count = search_places([ end_lat , end_long] , length)['Hospitals']
    e_fls_count = search_places([ end_lat , end_long] , length)['Fuel station']
    e_parks_count = search_places([ end_lat , end_long] , length)['Parks']
    e_schools_count = search_places([ end_lat , end_long] , length)['Schools']


    # Calculate places count in the start coordination 
    s_resturants_count = search_places([ start_lat , start_long] , length)['Resturants and Cafe\'s']
    s_college_count = search_places([ start_lat , start_long] , length)['College and University']
    s_hospitals_count = search_places([ start_lat , start_long] , length)['Hospitals']
    s_fls_count = search_places([ start_lat , start_long] , length)['Fuel station']
    s_parks_count = search_places([ start_lat , start_long] , length)['Parks']
    s_schools_count = search_places([ start_lat , start_long] , length)['Schools']



    main_street = encoded_street.get(main_street,0)
    start_street = bystreet_encoded.get(start_street,0)
    end_street = end_bystreet_encoded.get(end_street,0)
    day_in_week = days.get(day_in_week,0)
    


    inputs = [ main_street, start_street, end_street, day_in_week, length, time_taken, bus_count, start_lat, start_long, end_lat, end_long, 
          min , 
          hour,
          month,
          day_in_month,
          e_resturants_count,
          e_college_count,
          e_hospitals_count,
          e_fls_count,
          e_parks_count,
          e_schools_count,
          s_resturants_count,
          s_college_count,
          s_hospitals_count,
          s_fls_count,
          s_parks_count,
          s_schools_count]
    
    inputs = scaler.transform([inputs])
    
    # Make a prediction
    output = final_chosen_model.predict(np.array(inputs).reshape( 1, -1))[0]

    output = y_normalize.inverse_transform(output.reshape(-1, 1))[0][0]

    return f'{output:,.2f} mph\nAverage error : {result2.mean()[0]:.4f}'


interface = gr.Interface(
    fn=predict_congestion,

    examples=[
        ["Halsted", "51st", "55th", "Monday", 5, 30, 2, 40.7128, -74.0060, 40.7489, -73.9680],
        ["Cicero", "Kedzie", "Madison", "Tuesday", 8, 45, 5, 40.7559, -73.9864, 40.7590, -73.9845],
        ["Irving Park", "Western", "103rd", "Wednesday", 10, 50, 3, 34.0928, -118.3617, 34.0724, -118.3593],
    ],
    inputs=inputs,
    outputs=output,

    title="Traffic Jam Predictor",
    description="ML tool to predict the most likely areas to be congested.<br>\
                Output will represent the predicted speed in mile per hour which indicates the congestion level",
    # live=True,
    examples_in_scrollable=True,

)

interface.launch( share=True )

<ipython-input-31-a90736edcc0a>:63: UserWarning: You have unused kwarg parameters in Interface, please remove them: {'examples_in_scrollable': True}
  interface = gr.Interface(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://22e277d119612a5bcc.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces
